In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT_ID = os.getenv("LANGCHAIN_PROJECT_ID")

지원되는 모델 목록

<table>

<thead>
<tr>
<th>MODEL</th>
<th>PAGES PER DOLLAR</th>
<th>PERFORMANCE ON MTEB EVAL</th>
<th>MAX INPUT</th>
</tr>
</thead>
<tbody>
<tr>
<td>text-embedding-3-small</td>
<td>62,500</td>
<td>62.3%</td>
<td>8191</td>
</tr>
<tr>
<td>text-embedding-3-large</td>
<td>9,615</td>
<td>64.6%</td>
<td>8191</td>
</tr>
<tr>
<td>text-embedding-ada-002</td>
<td>12,500</td>
<td>61.0%</td>
<td>8191</td>
</tr>
</tbody>
</table>

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [3]:
text = "임베딩 테스트를 위한 샘플 문장입니다."

## 1. Quary Embedding

In [4]:
quary_result=embeddings.embed_query(text)

In [5]:
len(quary_result)

1536

## 2. Document embeddings

In [6]:
text_2 = "임베딩이란 고차원 데이터를 저차원 벡터로 변환하여 데이터의 의미와 관계를 효율적으로 표현하는 기술이다."

doc_result = embeddings.embed_documents(
    [text, text, text, text])

In [7]:
doc_result[0][:5]

[-0.012485523708164692,
 0.04285658523440361,
 0.01653285138309002,
 -0.0251642893999815,
 0.013387315906584263]

## 차원 지정

In [8]:
len(doc_result[0])

1536

In [9]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화합니다.
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [10]:
# 주어진 텍스트를 임베딩하고 첫 번째 임베딩 벡터의 길이를 반환합니다.
len(embeddings_1024.embed_documents([text])[0])

1024

## Similarity

In [11]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]

In [14]:
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [15]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [16]:
# sentence1 = "안녕하세요? 반갑습니다."
# sentence2 = "안녕하세요? 만나서 반가워요."
# sentence3 = "Hi, nice to meet you."
# sentence4 = "I like to eat apples."

for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8376] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5042] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1362] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.8142] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4790] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1318] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.5128] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1409] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2249] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.
